In [2]:
%load_ext nvcc4jupyter

from nvcc4jupyter import set_defaults
set_defaults(compiler_args='-arch=sm_100a -Xptxas=-v -O0 -I/workspace/cutlass/include')

Source files will be saved in "/tmp/tmpwohn4s39".


In [5]:
%%cuda

#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>
#include <cuda.h>
#include <cuda_fp16.h>
#include <stdint.h>
#include <cuda_bf16.h>

#define allocate_tmem(smem_dst, nCols) \ 
  asm volatile("tcgen05.alloc.cta_group::1.sync.aligned{.shared::cta}.b32  [%0], %1"); \
    :                                                                              \ 
    : "r"(smem_dst), "r"(nCols) \
    : "memory");\
               
#define deallocate_tmem(taddr, nCols) \ 
  asm volatile("tcgen05.dealloc.cta_group::1.sync.aligned.b32    [%0], %1"); \
    :                                                                        \ 
    : "r"(taddr), "r"(nCols) \
    : "memory");\
               
constexpr int N_cols = 32; 

__global__ void alloc_dealloc_tmem()
{
  __shared__ uint32_t A[1]; 
  uint32_t smem_addr = (uint32_t) &A[0];
  uint32_t n_cols = N_cols;
  allocate_tmem(smem_addr, n_cols); 
  uint32_t taddr = A[0]; 
  deallocate_tmem(taddr, n_cols);
  
}


int main()
{
  alloc_dealloc_tmem<<<1,32>>>();
  cudaDeviceSynchronize();
  return 0;
}

/tmp/tmpwohn4s39/4b175c7f-9c06-43d9-b968-704326085277/single_file.cu:10:40: warning: backslash and newline separated by space
   10 | #define allocate_tmem(smem_dst, nCols) \
      |                                         
/tmp/tmpwohn4s39/4b175c7f-9c06-43d9-b968-704326085277/single_file.cu:12:84: warning: backslash and newline separated by space
   12 |     :                                                                              \
      |                                                                                     
/tmp/tmpwohn4s39/4b175c7f-9c06-43d9-b968-704326085277/single_file.cu:16:39: warning: backslash and newline separated by space
   16 | #define deallocate_tmem(taddr, nCols) \
      |                                        
/tmp/tmpwohn4s39/4b175c7f-9c06-43d9-b968-704326085277/single_file.cu:18:78: warning: backslash and newline separated by space
   18 |     :                                                                        \
      |                      

Ok this is so fucked and I am so pissed off right now. 

In [14]:
l = 37 
c = 53 
def get_tmem_addr(lane, col): 
  return (c | (l<< 16))

def get_2d_from_TMEM_addr(tmem_addr): 
  c_mask = (1 << 16) - 1 
  col = tmem_addr & c_mask 
  row = tmem_addr >> 16
  return row, col
  
l_new, c_new  = get_2d_from_TMEM_addr(get_tmem_addr(l,c))
print(l_new, c_new)

tmem_addr = get_tmem_addr(l,c)
print(f"{tmem_addr:0{32}b}")

def print_bin(n): 
  print(f"{n:0{32}b}")

37 53
00000000001001010000000000110101


In [15]:
print_bin(tmem_addr % 2**16)

00000000000000000000000000110101


In [16]:
print_bin(tmem_addr // (2**16))

00000000000000000000000000100101


In [10]:
%%cuda

#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>
#include <cuda.h>
#include <cuda_fp16.h>
#include <stdint.h>
#include <cuda_bf16.h>


#define allocate_tmem(smem_dst, nCols) \
  asm volatile("tcgen05.alloc.cta_group::1.sync.aligned.shared::cta.b32  [%0], %1" \
    : /* no outputs */ \
    : "r"(smem_dst), "r"(nCols) \
    : "memory")

// FIX 1: Removed trailing whitespace and internal semicolons
// FIX 3: Changed constraint for taddr (now taddr_ptr) from "r" to "l"
#define deallocate_tmem(taddr_ptr, nCols) \
  asm volatile("tcgen05.dealloc.cta_group::1.sync.aligned.b32    %0, %1" \
    : /* no outputs */ \
    : "r"(taddr_ptr), "r"(nCols) \
    : "memory")

constexpr int N_cols = 32;

__global__ void alloc_dealloc_tmem()
{
  __shared__ uint32_t A[1];
  

  uint32_t smem_addr = (uint32_t) &A[0];
  uint32_t n_cols = N_cols;
  
  allocate_tmem(smem_addr, n_cols);

  uint32_t taddr = A[0];
  

  deallocate_tmem(taddr, n_cols);
}


int main()
{
  alloc_dealloc_tmem<<<1,32>>>();
  cudaDeviceSynchronize();
  printf("Kernel executed successfully.\n"); // Added a print for confirmation
  return 0;
}

/tmp/tmpwohn4s39/b4d84de6-380d-44ca-8576-33a101d68ddd/single_file.cu(32): warning #767-D: conversion from pointer to smaller integer
    uint32_t smem_addr = (uint32_t) &A[0];
                         ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

/tmp/tmpwohn4s39/b4d84de6-380d-44ca-8576-33a101d68ddd/single_file.cu(32): warning #767-D: conversion from pointer to smaller integer
    uint32_t smem_addr = (uint32_t) &A[0];
                         ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

/tmp/tmpwohn4s39/b4d84de6-380d-44ca-8576-33a101d68ddd/single_file.cu(32): warning #767-D: conversion from pointer to smaller integer
    uint32_t smem_addr = (uint32_t) &A[0];
                         ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

ptxas /tmp/tmpxft_00000858_00000000-6_single_file.compute_100a.ptx, line 31; fatal   : Parsing error near 'ld': syntax error
ptxas fatal   : Ptx assemb

KEEP THE BELOW CODE IT IS PRECIOUS 

In [ ]:
%%cuda

#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>
#include <cuda.h>
#include <cuda_fp16.h>
#include <stdint.h>
#include <cuda_bf16.h>

#define allocate_tmem(smem_dst_ptr, nCols) \
  asm volatile("tcgen05.alloc.cta_group::1.sync.aligned.shared::cta.b32  [%0], %1;" \
    : /* no outputs */ \
    : "l"(smem_dst_ptr), "r"(nCols) \
    : "memory")


#define deallocate_tmem(taddr_handle, nCols) \
  asm volatile("tcgen05.dealloc.cta_group::1.sync.aligned.b32    %0, %1;" \
    : /* no outputs */ \
    : "r"(taddr_handle), "r"(nCols) \
    : "memory")

constexpr int N_cols = 32;

__global__ void alloc_dealloc_tmem()
{
  __shared__ uint32_t A[1]; 
  
  uint64_t smem_addr = (uint64_t) &A[0];
  uint32_t n_cols = N_cols;
  

  allocate_tmem(smem_addr, n_cols); 
  
  uint32_t taddr = A[0]; 
  deallocate_tmem(taddr, n_cols);
}


int main()
{
  alloc_dealloc_tmem<<<1,32>>>();
  cudaDeviceSynchronize();
  printf("Kernel executed successfully.\n");
  return 0;
}

Kernel executed successfully.



KEEP THE ABOVE CODE IT IS PRECIOUS 

In [21]:
%%cuda

#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>
#include <cuda.h>
#include <cuda_fp16.h>
#include <stdint.h>
#include <cuda_bf16.h>

#define allocate_tmem(smem_dst_ptr, nCols) \
  asm volatile("tcgen05.alloc.cta_group::1.sync.aligned.shared::cta.b32  [%0], %1;" \
    : /* no outputs */ \
    : "l"(smem_dst_ptr), "r"(nCols) \
    : "memory")


#define deallocate_tmem(taddr_handle, nCols) \
  asm volatile("tcgen05.dealloc.cta_group::1.sync.aligned.b32    %0, %1;" \
    : /* no outputs */ \
    : "r"(taddr_handle), "r"(nCols) \
    : "memory")

constexpr int N_cols = 32;

__global__ void alloc_dealloc_tmem()
{
  int t = threadIdx.x; 
  
  __shared__ uint32_t A[1]; 
  
  uint64_t smem_addr = (uint64_t) &A[0];
  uint32_t n_cols = N_cols;
  

  allocate_tmem(smem_addr, n_cols); 
  
  uint32_t taddr = A[0]; 
  //#threads have arrived here
  __syncthreads();
  if (t == 0)
  {
    printf("This is the start of the address in TMEM, as stored in SMEM after allocation: %d \n", taddr);
    
  }
  __syncthreads();
  
  deallocate_tmem(taddr, n_cols);

  
}


int main()
{
  alloc_dealloc_tmem<<<1,32>>>();
  cudaDeviceSynchronize();
  printf("Kernel executed successfully.\n");
  return 0;
}

Kernel executed successfully.



In [25]:
%%cuda

#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>
#include <cuda.h>
#include <cuda_fp16.h>
#include <stdint.h>
#include <cuda_bf16.h>

#define allocate_tmem(smem_dst_ptr, nCols) \
  asm volatile("tcgen05.alloc.cta_group::1.sync.aligned.shared::cta.b32  [%0], %1;" \
    : /* no outputs */ \
    : "l"(smem_dst_ptr), "r"(nCols) \
    : "memory")


#define deallocate_tmem(taddr_handle, nCols) \
  asm volatile("tcgen05.dealloc.cta_group::1.sync.aligned.b32    %0, %1;" \
    : /* no outputs */ \
    : "r"(taddr_handle), "r"(nCols) \
    : "memory")

constexpr int N_cols = 32;

__global__ void alloc_dealloc_tmem(uint32_t* d_debug_output) // <-- Pass in a global pointer
{
  int t = threadIdx.x; 
  
  __shared__ uint32_t A[1]; 
  A[0] = 100;
  
  uint64_t smem_addr = (uint64_t) &A[0];
  uint32_t n_cols = N_cols;
  
  allocate_tmem(smem_addr, n_cols); 
  
  uint32_t taddr = A[0]; 


  deallocate_tmem(taddr, n_cols);
  
  if (t == 0)
  {
    *d_debug_output = A[0]; // Write the handle to global memory
  }

}


int main()
{

  uint32_t* d_debug_val;
  cudaMalloc(&d_debug_val, sizeof(uint32_t));

  uint32_t h_debug_val = 12;

  alloc_dealloc_tmem<<<1,32>>>(d_debug_val);
  
  cudaError_t err = cudaDeviceSynchronize();
  if (err != cudaSuccess) {
      printf("Kernel failed: %s\n", cudaGetErrorString(err));
  }

  cudaMemcpy(&h_debug_val, d_debug_val, sizeof(uint32_t), cudaMemcpyDeviceToHost);

  printf("Kernel executed successfully.\n");
  printf("This is the start of the address in TMEM: %u \n", h_debug_val);
  cudaFree(d_debug_val);
  return 0;
}

Kernel executed successfully.
This is the start of the address in TMEM: 0 



if every warp can allocate 32 columns of tmem and store its start address, then we can deploy 2 warps, and make them 
allocate different bits of tmem and see what happens.

In [ ]:
%%cuda

#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>
#include <cuda.h>
#include <cuda_fp16.h>
#include <stdint.h>
#include <cuda_bf16.h>
#include <time.h>



#define allocate_tmem(smem_dst_ptr, nCols) \
  asm volatile("tcgen05.alloc.cta_group::1.sync.aligned.shared::cta.b32  [%0], %1;" \
    : /* no outputs */ \
    : "l"(smem_dst_ptr), "r"(nCols) \
    : "memory")


#define deallocate_tmem(taddr_handle, nCols) \
  asm volatile("tcgen05.dealloc.cta_group::1.sync.aligned.b32    %0, %1;" \
    : /* no outputs */ \
    : "r"(taddr_handle), "r"(nCols) \
    : "memory")

constexpr int N_cols = 32;
constexpr int N_ctas = 1; 
constexpr int N_warps = 2; 
constexpr int N_threads_per_block = N_warps*32;

__global__ void alloc_dealloc_tmem(uint32_t* d_debug_output)
{
  int t = threadIdx.x; 
  int warp_id = t / 32;
  int lane_id = t % 32;
  
  __shared__ uint32_t A[N_warps]; 
  
  uint64_t smem_addr = (uint64_t) &A[warp_id];
  uint32_t n_cols = N_cols;
  
  allocate_tmem(smem_addr, n_cols); 
  
  uint32_t taddr = A[warp_id]; 

  deallocate_tmem(taddr, n_cols);
  
  if (lane_id == 0)
  {
    d_debug_output[warp_id] = A[warp_id];
  }
}


int main()
{

  uint32_t* d_debug_val;
  uint32_t* h_debug_val;
  size_t size = N_warps * sizeof(uint32_t);
  
  cudaMalloc(&d_debug_val, size);
  h_debug_val = (uint32_t*)malloc(size);

  srand(time(NULL));
  printf("Host array before kernel (random values):\n");
  for (int i = 0; i < N_warps; i++) {
    h_debug_val[i] = rand();
    printf("h_debug_val[%d] = %u\n", i, h_debug_val[i]);
  }

  alloc_dealloc_tmem<<<N_ctas,N_threads_per_block>>>(d_debug_val);
  
  cudaError_t err = cudaDeviceSynchronize();
  if (err != cudaSuccess) {
      printf("Kernel failed: %s\n", cudaGetErrorString(err));
  }

  cudaMemcpy(h_debug_val, d_debug_val, size, cudaMemcpyDeviceToHost);

  printf("\nKernel executed successfully.\n");
  printf("TMEM addresses from device:\n");
  for (int i = 0; i < N_warps; i++) {
    printf("Warp %d TMEM address: %u \n", i, h_debug_val[i]);
  }

  cudaFree(d_debug_val);
  free(h_debug_val);
  return 0;
}

Host array before kernel (random values):
h_debug_val[0] = 1391302528
h_debug_val[1] = 1093967198

Kernel executed successfully.
TMEM addresses from device:
Warp 0 TMEM address: 0 
Warp 1 TMEM address: 0 



In [31]:
%%cuda

#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>
#include <cuda.h>
#include <cuda_fp16.h>
#include <stdint.h>
#include <cuda_bf16.h>
#include <time.h>



#define allocate_tmem(smem_dst_ptr, nCols) \
  asm volatile("tcgen05.alloc.cta_group::2.sync.aligned.shared::cta.b32  [%0], %1;" \
    : /* no outputs */ \
    : "l"(smem_dst_ptr), "r"(nCols) \
    : "memory")


#define deallocate_tmem(taddr_handle, nCols) \
  asm volatile("tcgen05.dealloc.cta_group::2.sync.aligned.b32    %0, %1;" \
    : /* no outputs */ \
    : "r"(taddr_handle), "r"(nCols) \
    : "memory")

constexpr int N_cols = 32;
constexpr int N_ctas = 2; 
constexpr int N_warps = 1; 
constexpr int N_threads_per_block = N_warps*32;

__global__ void alloc_dealloc_tmem(uint32_t* d_debug_output)
{
  
  int b = blockIdx.x; 
  
  __shared__ uint32_t A[1]; 
  
  uint64_t smem_addr = (uint64_t) &A[0];
  uint32_t n_cols = N_cols;
  
  allocate_tmem(smem_addr, n_cols); 
  
  uint32_t taddr = A[0]; 

  deallocate_tmem(taddr, n_cols);
  

  d_debug_output[b] = A[0];
  
}


int main()
{

  uint32_t* d_debug_val;
  uint32_t* h_debug_val;
  size_t size = N_ctas * sizeof(uint32_t);
  
  cudaMalloc(&d_debug_val, size);
  h_debug_val = (uint32_t*)malloc(size);

  srand(time(NULL));
  printf("Host array before kernel (random values):\n");
  for (int i = 0; i < N_ctas; i++) {
    h_debug_val[i] = rand();
    printf("h_debug_val[%d] = %u\n", i, h_debug_val[i]);
  }

  alloc_dealloc_tmem<<<N_ctas,N_threads_per_block>>>(d_debug_val);
  
  cudaError_t err = cudaDeviceSynchronize();
  if (err != cudaSuccess) {
      printf("Kernel failed: %s\n", cudaGetErrorString(err));
  }

  cudaMemcpy(h_debug_val, d_debug_val, size, cudaMemcpyDeviceToHost);

  printf("\nKernel executed successfully.\n");
  printf("TMEM addresses from device:\n");
  for (int i = 0; i < N_ctas; i++) {
    printf("CTA %d TMEM address: %u \n", i, h_debug_val[i]);
  }

  cudaFree(d_debug_val);
  free(h_debug_val);
  return 0;
}

Host array before kernel (random values):
h_debug_val[0] = 771223414
h_debug_val[1] = 181268272

Kernel executed successfully.
TMEM addresses from device:
CTA 0 TMEM address: 0 
CTA 1 TMEM address: 0 



In [36]:
%%cuda

#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>
#include <cuda.h>
#include <cuda_fp16.h>
#include <stdint.h>
#include <cuda_bf16.h>
#include <time.h>



#define allocate_tmem(smem_dst_ptr, nCols) \
  asm volatile("tcgen05.alloc.cta_group::1.sync.aligned.shared::cta.b32  [%0], %1;" \
    : /* no outputs */ \
    : "l"(smem_dst_ptr), "r"(nCols) \
    : "memory")


#define deallocate_tmem(taddr_handle, nCols) \
  asm volatile("tcgen05.dealloc.cta_group::1.sync.aligned.b32    %0, %1;" \
    : /* no outputs */ \
    : "r"(taddr_handle), "r"(nCols) \
    : "memory")

constexpr int N_cols = 32;
constexpr int N_ctas = 1; 
constexpr int N_warps = 1; 
constexpr int N_threads_per_block = N_warps*32;
  constexpr int N_debug_vals = 4;

__global__ void alloc_dealloc_tmem(uint32_t* d_debug_output)
{
  
  int b = blockIdx.x; 
  int t = threadIdx.x;
  
  __shared__ uint32_t A[2]; 
  
  uint64_t smem_addr_0 = (uint64_t) &A[0];
  uint64_t smem_addr_1 = (uint64_t) &A[1];
  uint32_t n_cols = N_cols;
  
  
  allocate_tmem(smem_addr_0, n_cols); 
  
  allocate_tmem(smem_addr_1, n_cols);
  __syncthreads();
  
    if (t == 0)
  {
    d_debug_output[2*b + 0] = A[0]; 
    d_debug_output[2*b + 1] = A[1]; 
  }
  __syncthreads();
  uint32_t taddr_0 = A[0]; 
  uint32_t taddr_1 = A[1];

  deallocate_tmem(taddr_0, n_cols);
  deallocate_tmem(taddr_1, n_cols);
  


  
}


int main()
{


  uint32_t* d_debug_val;
  uint32_t* h_debug_val;
  size_t size = N_debug_vals * sizeof(uint32_t);
  
  cudaMalloc(&d_debug_val, size);
  h_debug_val = (uint32_t*)malloc(size);

  srand(time(NULL));
  printf("Host array before kernel (random values):\n");
  for (int i = 0; i < N_debug_vals; i++) {
    h_debug_val[i] = rand();
    printf("h_debug_val[%d] = %u\n", i, h_debug_val[i]);
  }
  cudaMemcpy(d_debug_val, h_debug_val, size, cudaMemcpyHostToDevice);
  alloc_dealloc_tmem<<<N_ctas,N_threads_per_block>>>(d_debug_val);
  
  cudaError_t err = cudaDeviceSynchronize();
  if (err != cudaSuccess) {
      printf("Kernel failed: %s\n", cudaGetErrorString(err));
  }

  cudaMemcpy(h_debug_val, d_debug_val, size, cudaMemcpyDeviceToHost);

  printf("\nKernel executed successfully.\n");
  printf("TMEM addresses from device:\n");
  for (int i = 0; i < N_debug_vals; i++) {
    printf("CTA %d TMEM address: %u \n", i, h_debug_val[i]);
  }

  cudaFree(d_debug_val);
  free(h_debug_val);
  return 0;
}

Host array before kernel (random values):
h_debug_val[0] = 934915272
h_debug_val[1] = 634184069
h_debug_val[2] = 1806837172
h_debug_val[3] = 1144539932

Kernel executed successfully.
TMEM addresses from device:
CTA 0 TMEM address: 934915272 
CTA 1 TMEM address: 634184069 
CTA 2 TMEM address: 1806837172 
CTA 3 TMEM address: 1144539932 



Ah now we see the issue, the allocation isn't even happening. 